In [3]:
from sympy import *

In [4]:
c, s, D, d, o, t = symbols('c s D d o t')

In [5]:
def s_t(y):
    return s*exp(-c*y)

In [21]:
i = integrate(s_t(o+t*d)*D, (t, 0, 1))

In [22]:
i

Piecewise((-D*s*exp(-c*(d + o))/(c*d) + D*s*exp(-c*o)/(c*d), Ne(c*d, 0)), (D*s, True))

In [23]:
simplify(i)

Piecewise((D*s*(exp(c*d) - 1)*exp(-c*(d + o))/(c*d), Ne(c*d, 0)), (D*s, True))

In [27]:
crytekResult = s*exp(-c*o) * D * (1-exp(-c*d))/(c*d)

In [33]:
simplify(crytekResult)

D*s*(exp(c*d) - 1)*exp(-c*(d + o))/(c*d)